In [ ]:
# if the package is not installed in your python environment, run this to execute the notebook directly from inside the GitHub repository
%cd -q ..

In [ ]:
# some python imports that will be used throughout the tutorial
import numpy as np
import pandas as pd
import pint
import xarray as xr

In [ ]:
# importing the weldx package with prevalent default abbreviations
import weldx
import weldx.geometry as geo
import weldx.transformations as tf
import weldx.utility as ut
import weldx.visualization as vis
from weldx import Q_

### groove shape

In [ ]:
from weldx.welding.groove.iso_9692_1 import get_groove

groove = get_groove(
    groove_type="VGroove",
    workpiece_thickness=Q_(0.5, "cm"),
    groove_angle=Q_(50, "deg"),
    root_face=Q_(1, "mm"),
    root_gap=Q_(1, "mm"),
)

### workpiece geometry

In [ ]:
# define the weld seam length in mm
seam_length = 100

# create a linear trace segment a the complete weld seam trace
trace_segment = geo.LinearHorizontalTraceSegment(seam_length)
trace = geo.Trace(trace_segment)

# create 3d workpiece geometry from the groove profile and trace objects
geometry = geo.Geometry(groove.to_profile(width_default=Q_(50, "mm")), trace)

# rasterize geometry
profile_raster_width = 0.5  # resolution of each profile in mm
trace_raster_width = 15  # space between profiles in mm
geometry_data_sp = geometry.rasterize(
    profile_raster_width=profile_raster_width, trace_raster_width=trace_raster_width
)

## Test rasterization

In [ ]:
geos = geometry.rasterize(
    profile_raster_width=profile_raster_width,
    trace_raster_width=trace_raster_width,
    stack=False,
)

In [ ]:
def triangulate_geometry(geo_data):
    nx = geo_data.shape[2]  # Points per profile
    ny = geo_data.shape[0]  # number of profiles

    data = np.swapaxes(geo_data, 1, 2).reshape((-1, 3))
    triangle_indices = np.empty((ny - 1, nx - 1, 2, 3), dtype=int)
    r = np.arange(nx * ny).reshape(ny, nx)
    triangle_indices[:, :, 0, 0] = r[:-1, :-1]
    triangle_indices[:, :, 1, 0] = r[:-1, 1:]
    triangle_indices[:, :, 0, 1] = r[:-1, 1:]

    triangle_indices[:, :, 1, 1] = r[1:, 1:]
    triangle_indices[:, :, :, 2] = r[1:, :-1, None]
    triangle_indices.shape = (-1, 3)

    return data, triangle_indices

In [ ]:
data, triangle_indices = triangulate_geometry(geos[0])
print(data.shape)
print(triangle_indices)

In [ ]:
from ipygany import Component, IsoColor, PolyMesh, Scene, Threshold, Warp


def ipygany_meshes(geo_list):
    meshes = []
    for geo_data in geo_list:
        data, triangle_indices = triangulate_geometry(geo_data)
        meshes.append(
            PolyMesh(
                vertices=data,
                triangle_indices=triangle_indices,
            )
        )
    return meshes

In [ ]:
meshes = ipygany_meshes(geos)
Scene(meshes)